In [1]:
from pathlib import Path
import os,sys, json, datetime, getopt
import pickle as pkl
import time

In [2]:

oakPaths = {}
oakPaths['Root'] = Path('/mnt/o/giocomo/alexg/')
oakPaths['Clustered'] = Path('/mnt/o/giocomo/alexg/Clustered/')
oakPaths['PreProcessed'] = Path('/mnt/o/giocomo/alexg/PreProcessed/')
oakPaths['Raw'] = Path('/mnt/o/giocomo/alexg/RawData/InVivo/')
oakPaths['Analyses'] = Path('/mnt/o/giocomo/alexg/Analyses')


In [3]:
def getSessionPaths(rootPath, session,step=0.02,SR=32000):
    tmp = session.split('_')
    animal = tmp[0]
    task = tmp[1]
    date = tmp[2]

    Paths = {}
    Paths['session'] = session
    Paths['animal']=animal
    Paths['task'] = task
    Paths['date'] = date
    Paths['step'] = step
    Paths['SR'] = SR
    Paths['Clusters'] = rootPath['Clustered'] / animal /(session+'_KSClusters')
    Paths['Raw'] = rootPath['Raw'] / animal / session
    Paths['PreProcessed'] = rootPath['PreProcessed'] / animal / (session + '_Results')
    Paths['Analyses'] = rootPath['Analyses'] / animal/ (session + '_Analyses')

    Paths['ClusterTable'] = rootPath['Clustered'] / animal / (animal+'_ClusteringSummary.json')

    if not Paths['Clusters'].exists():
        print('Error, no Cluster Folder found.')
    if not Paths['PreProcessed'].exists():
        print('Error, no processed binaries found.')
    if not Paths['ClusterTable'].exists():
        print('Error, no clustering table found.')

    Paths['Analyses'].mkdir(parents=True, exist_ok=True)

    Paths['BehavTrackDat'] = Paths['Analyses'] / ('BehTrackVariables_{}ms.h5'.format(int(step*1000)))

    Paths['Cell_Spikes'] = Paths['Analyses'] / 'Cell_Spikes.json'
    Paths['Cell_Bin_Spikes'] = Paths['Analyses'] / ('Cell_Bin_Spikes_{}ms.npy'.format(int(step*1000)))
    Paths['Cell_FR'] = Paths['Analyses'] / ('Cell_FR_{}ms.npy'.format(int(step*1000)))

    Paths['Mua_Spikes'] = Paths['Analyses'] / 'Mua_Spikes.json'
    Paths['Mua_Bin_Spikes'] = Paths['Analyses'] / ('Mua_Bin_Spikes_{}ms.npy'.format(int(step*1000)))
    Paths['Mua_FR'] = Paths['Analyses'] / ('Mua_FR_{}ms.npy'.format(int(step*1000)))

    Paths['Spike_IDs'] = Paths['Analyses'] / 'Spike_IDs.json'
    Paths['ZoneAnalyses'] = Paths['Analyses'] / 'ZoneAnalyses.pkl'

    Paths['TrialInfo'] = Paths['Analyses'] / 'TrInfo.pkl'
    Paths['TrialCondMat'] = Paths['Analyses'] / 'TrialCondMat.csv'
    Paths['TrLongPosMat'] = Paths['Analyses'] / 'TrLongPosMat.csv'
    Paths['TrLongPosFRDat'] = Paths['Analyses'] / 'TrLongPosFRDat.csv'
    Paths['TrModelFits'] = Paths['Analyses'] /  'TrModelFits.csv'

    # plots directories
    Paths['Plots'] = Paths['Analyses'] / 'Plots'
    Paths['Plots'].mkdir(parents=True, exist_ok=True)
    Paths['SampCountsPlots'] = Paths['Plots'] / 'SampCountsPlots'
    Paths['SampCountsPlots'].mkdir(parents=True, exist_ok=True)

    Paths['ZoneFRPlots'] = Paths['Plots'] / 'ZoneFRPlots'
    Paths['ZoneFRPlots'].mkdir(parents=True, exist_ok=True)

    Paths['ZoneCorrPlots'] = Paths['Plots'] / 'ZoneCorrPlots'
    Paths['ZoneCorrPlots'].mkdir(parents=True, exist_ok=True)
    Paths['SIPlots'] = Paths['Plots'] / 'SIPlots'
    Paths['SIPlots'].mkdir(parents=True, exist_ok=True)

    Paths['TrialPlots'] = Paths['Plots'] / 'TrialPlots'
    Paths['TrialPlots'].mkdir(parents=True, exist_ok=True)


In [9]:
an = 'Li'
Tasks = ['OF','T3g','T3h','T3j','T3i']

In [7]:
oakPaths['Raw']/an

PosixPath('/mnt/o/giocomo/alexg/RawData/InVivo/Li')

In [17]:
sessions = []
if (oakPaths['Raw']/an).exists():
    for session in (oakPaths['Raw']/an).glob('*'):
        try:
            temp = session.name.split('_')
            anID = temp[0]
            task = temp[1]
            date = temp[2]

            if (anID==an) & (task in Tasks):
                sessions.append(session.name)
        except:
            pass


In [18]:
sessions

['Li_OF_052318',
 'Li_OF_052418',
 'Li_OF_052518',
 'Li_OF_052818',
 'Li_OF_052918',
 'Li_OF_053018',
 'Li_OF_053118',
 'Li_OF_060118',
 'Li_OF_060418',
 'Li_OF_060518',
 'Li_OF_060718',
 'Li_OF_060818',
 'Li_OF_061118',
 'Li_OF_061218',
 'Li_OF_061318',
 'Li_OF_061418',
 'Li_OF_061518',
 'Li_OF_061818',
 'Li_OF_062018',
 'Li_OF_062518',
 'Li_OF_062618',
 'Li_OF_062718',
 'Li_OF_062818',
 'Li_OF_062918',
 'Li_OF_070218',
 'Li_OF_070618',
 'Li_OF_071018',
 'Li_OF_080118',
 'Li_OF_080218',
 'Li_T3g_052418',
 'Li_T3g_052518',
 'Li_T3g_052818',
 'Li_T3g_052918',
 'Li_T3g_053018',
 'Li_T3g_053118',
 'Li_T3g_060118',
 'Li_T3g_060418',
 'Li_T3g_061318',
 'Li_T3g_061418',
 'Li_T3g_061518',
 'Li_T3g_061818',
 'Li_T3g_062018',
 'Li_T3g_062518',
 'Li_T3g_062618',
 'Li_T3g_062718',
 'Li_T3g_062818',
 'Li_T3g_062918',
 'Li_T3g_070218',
 'Li_T3g_070518',
 'Li_T3g_070618',
 'Li_T3g_070918',
 'Li_T3g_071018',
 'Li_T3g_071118',
 'Li_T3g_072518',
 'Li_T3g_072618',
 'Li_T3g_073018',
 'Li_T3g_080118',
 'L